In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip


In [2]:
# /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip

In [3]:
# !git clone https://github.com/Muhammad-Shah/Toxity-Classifier-App.git
# !git status
# !git add --all
!git config --global user.email ""
!git config --global user.name ""
# !git commit -m "testing"
!git push

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [4]:
%cd /kaggle/working/Toxity-Classifier-App

[Errno 2] No such file or directory: '/kaggle/working/Toxity-Classifier-App'
/kaggle/working


In [2]:
import re
import pandas as pd
import numpy as np
import spacy
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import nltk
from spacy import load
nltk.download('omw-1.4')
nltk.download('wordnet2022')
nlp = load('en_core_web_sm')

! cp -rf /usr/share/nltk_data/corpora/wordnet2022 /usr/share/nltk_data/corpora/wordnet # temp fix for lookup error.


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package wordnet2022 to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/wordnet2022.zip.


# Toxity Classifier NLP Project
=====================================

## 1. Introduction
---------------

### 1.1 Project Overview

* This project aims to develop a toxicity classifier to identify toxic comments online.
* The proliferation of toxic comments online has significant negative impacts on individuals and communities, necessitating the development of effective detection and mitigation strategies.

### 1.2 Problem Statement

* Classify text as toxic or non-toxic to prevent online harassment and promote respectful communication.
* To create a safer and more inclusive online environment, free from hate speech and cyberbullying.

## 2. Data Exploration
---------------------

### 2.1 Data Loading

#### 2.1.1 Load Dataset

* Load the dataset into a Pandas dataframe
* Display the first few rows of the data using `head()`

In [9]:
train_data = pd.read_csv(filepath_or_buffer='/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


#### 2.1.2 Data Source

* **Data Source:** Jigsaw Toxic Comment Classification Challenge on Kaggle
* **Data Collection:** The dataset was collected from Wikipedia comments and annotated by human raters for toxicity.
* **Data Description:** The dataset consists of a CSV file containing approximately 159,571 comments, each labeled as toxic or non-toxic.
* **Data Download:** The dataset can be downloaded from the Kaggle competition page: [/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip](/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip)

### 2.1.3 Dataset Description

You are provided with a large number of Wikipedia comments which have been labeled by human raters for toxic behavior. The types of toxicity are:

* toxic
* severe_toxic
* obscene
* threat
* insult
* identity_hate

`You must create a model which predicts a probability of each type of toxicity for each comment`

### 2.2 Data Summary

#### 2.2.1 Data Statistics

* Provide a summary of the data using `info()` and `describe()`
* Visualize the data distribution using plots (e.g., histograms, box plots)

In [10]:
train_data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [11]:
train_data.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### 2.2.2 Data Quality

* Check for missing values and handle them accordingly
* Check for duplicates and handle them accordingly

In [12]:
train_data.isna().sum()

id               0
comment_text     0
toxic            0
severe_toxic     0
obscene          0
threat           0
insult           0
identity_hate    0
dtype: int64

In [13]:
columns = list(train_data.columns[1:])
train_data.duplicated(subset=columns).value_counts()

False    159571
Name: count, dtype: int64

In [14]:
duplicates = train_data[train_data.duplicated(subset='comment_text', keep=False)]
print(f"Number of duplicates: {len(duplicates)}")
print(duplicates)

Number of duplicates: 0
Empty DataFrame
Columns: [id, comment_text, toxic, severe_toxic, obscene, threat, insult, identity_hate]
Index: []


In [15]:
for col in train_data.columns[2:]:
    print(f"Unique values in column {col}: {train_data[col].unique()}")

Unique values in column toxic: [0 1]
Unique values in column severe_toxic: [0 1]
Unique values in column obscene: [0 1]
Unique values in column threat: [0 1]
Unique values in column insult: [0 1]
Unique values in column identity_hate: [0 1]


In [16]:
for col in train_data.columns[2:]:
    print(f"The distribution of toxicity labels in column: {train_data[col].value_counts()}")

The distribution of toxicity labels in column: toxic
0    144277
1     15294
Name: count, dtype: int64
The distribution of toxicity labels in column: severe_toxic
0    157976
1      1595
Name: count, dtype: int64
The distribution of toxicity labels in column: obscene
0    151122
1      8449
Name: count, dtype: int64
The distribution of toxicity labels in column: threat
0    159093
1       478
Name: count, dtype: int64
The distribution of toxicity labels in column: insult
0    151694
1      7877
Name: count, dtype: int64
The distribution of toxicity labels in column: identity_hate
0    158166
1      1405
Name: count, dtype: int64


### 2.3 Data Quality Check

#### 2.3.1 Handle Missing Values

* Explain how missing values were handled
* Provide justification for the chosen method

In [17]:
# NO Missing values are found

#### 2.3.2 Handle Duplicates

* Explain how duplicates were handled
* Provide justification for the chosen method

In [18]:
# NO Duplicates are found

## 3. Data Preprocessing
----------------------

### 3.1 Text Preprocessing

Text preprocessing is the process of cleaning and transforming raw text data into a format that is suitable for analysis or modeling by removing unnecessary characters, converting all text to lowercase, removing stop words, and stemming or lemmatizing words to their base form.

In [19]:
# Analyze some text columns
random_comments = train_data['comment_text'].sample(n=10, random_state=42)
for comment in random_comments.values:
    print(comment)
    print('------' * 10)

Geez, are you forgetful!  We've already discussed why Marx  was  not an anarchist, i.e. he wanted to use a State to mold his 'socialist man.'  Ergo, he is a statist - the opposite of an  anarchist.  I know a guy who says that, when he gets old and his teeth fall out, he'll quit eating meat.  Would you call him a vegetarian?
------------------------------------------------------------
Carioca RFA 

Thanks for your support on my request for adminship.

The final outcome was (31/4/1), so I am now an administrator. If you have any comments or concerns on my actions as an administrator, please let me know. Thank you!
------------------------------------------------------------
"

 Birthday 

No worries, It's what I do ;)Enjoy ur day|talk|e "
------------------------------------------------------------
Pseudoscience category? 

I'm assuming that this article is in the pseudoscience category because of its association with creationism.  However, there are modern, scientifically-accepted varia

#### 3.1.1 Tokenization

* Tokenize the text data using `word_tokenize`
* Explain the importance of tokenization

In [20]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

#### 3.1.2 Stopword Removal

* Remove stop words using `stopwords`
* Explain the importance of stopword removal

In [21]:
stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

#### 3.1.3 Lemmatization

* Lemmatize words using `WordNetLemmatizer`
* Explain the importance of lemmatization

In [25]:
# NLTK
lemmatizer = WordNetLemmatizer()

def preprocess_1(sentence: str):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+',' ', sentence)
    sentence = word_tokenize(sentence)
    
    sentence = ' '.join(lemmatizer.lemmatize(word.lower()) for word in sentence if word.lower() not in stopwords)
    return sentence

In [26]:
# SPACY
def preprocess_2(sentence: str):
    sentence = re.sub(r"http\S+", "", sentence)
    sentence = decontracted(sentence)
    sentence = re.sub("\S*\d\S*", "", sentence).strip()
    sentence = re.sub('[^A-Za-z]+',' ', sentence)
    doc = nlp(sentence)
    lemmatized_tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(lemmatized_tokens)

In [28]:
# DEMO of comments after preprocessing 
# * Tokenize
# * Stopwords Removal
# * Lemmatize
sample_data = train_data.head()

result = sample_data['comment_text'].apply(lambda x: preprocess_2(x))

for sentence in result:
    print(sentence)
    print('------' * 10)

Explanation edit username Hardcore Metallica Fan revert vandalism closure gas vote New York Dolls FAC remove template talk page retire
------------------------------------------------------------
d aww match background colour seemingly stuck thank talk January UTC
------------------------------------------------------------
hey man try edit war guy constantly remove relevant information talk edit instead talk page care formatting actual info
------------------------------------------------------------
  real suggestion improvement wonder section statistic later subsection type accident think reference need tidying exact format ie date format etc later preference format style reference want let know appear backlog article review guess delay reviewer turn list relevant form eg Wikipedia good article nomination Transport
------------------------------------------------------------
sir hero chance remember page
------------------------------------------------------------


In [ ]:
# Demo
train_data['processed_comments'] = train_data['comment_text'].apply(lambda x: preprocess(x))

### 3.2 Feature Engineering

#### 3.2.1 TF-IDF

* Extract relevant features from the text data using TF-IDF
* Explain the importance of TF-IDF

In [35]:
# Demo
tf_idf = TfidfVectorizer()
tf_idf.fit(result)
print("some sample features(unique words in the corpus)",tf_idf.get_feature_names_out())
print('====='*30)

final_tf_idf = tf_idf.transform(result)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

some sample features(unique words in the corpus) ['accident' 'actual' 'appear' 'article' 'aww' 'background' 'backlog'
 'care' 'chance' 'closure' 'colour' 'constantly' 'date' 'delay' 'dolls'
 'edit' 'eg' 'etc' 'exact' 'explanation' 'fac' 'fan' 'form' 'format'
 'formatting' 'gas' 'good' 'guess' 'guy' 'hardcore' 'hero' 'hey' 'ie'
 'improvement' 'info' 'information' 'instead' 'january' 'know' 'later'
 'let' 'list' 'man' 'match' 'metallica' 'need' 'new' 'nomination' 'page'
 'preference' 'real' 'reference' 'relevant' 'remember' 'remove' 'retire'
 'revert' 'review' 'reviewer' 'section' 'seemingly' 'sir' 'statistic'
 'stuck' 'style' 'subsection' 'suggestion' 'talk' 'template' 'thank'
 'think' 'tidying' 'transport' 'try' 'turn' 'type' 'username' 'utc'
 'vandalism' 'vote' 'want' 'war' 'wikipedia' 'wonder' 'york']
the type of count vectorizer  <class 'scipy.sparse._csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (5, 85)
the number of unique words including both unigrams and bigrams  85


In [ ]:
tf_idf = TfidfVectorizer()
tf_idf.fit(train_data['processed_comments'])
print("some sample features(unique words in the corpus)",tf_idf.get_feature_names()[0:10])
print('='*50)

final_tf_idf = tf_idf.transform(train_data['processed_comments'])
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

#### 3.2.2 Word Embeddings

* Extract relevant features from the text data using word embeddings
* Explain the importance of word embeddings

## 4. Model Selection and Training
--------------------------------

### 4.1 Model Selection

#### 4.1.1 Algorithm Selection

* Select a suitable machine learning algorithm for the task (e.g., Logistic Regression, SVM, Random Forest)
* Justify the choice of algorithm based on the problem and data characteristics

#### 4.1.2 Hyperparameter Tuning

* Explain the importance of hyperparameter tuning
* Discuss the methods used for hyperparameter tuning (e.g., grid search, random search)

### 4.2 Model Training

#### 4.2.1 Data Split

* Split the data into training and testing sets using `train_test_split`
* Explain the importance of data splitting

#### 4.2.2 Model Training

* Train the selected model on the training data
* Explain the training process

## 5. Model Evaluation
---------------------

### 5.1 Model Performance Metrics

#### 5.1.1 Accuracy

* Evaluate the model using accuracy as a performance metric
* Explain the importance of accuracy

#### 5.1.2 F1-Score

* Evaluate the model using F1-score as a performance metric
* Explain the importance of F1-score

#### 5.1.3 ROC-AUC

* Evaluate the model using ROC-AUC as a performance metric
* Explain the importance of ROC-AUC

### 5.2 Model Comparison

#### 5.2.1 Model Comparison Metrics

* Compare the performance of different models using relevant metrics
* Explain the importance of model comparison

#### 5.2.2 Model Selection

* Select the best-performing model based on the comparison results
* Explain the justification for the chosen model

## 6. Model Deployment
---------------------

### 6.1 Model Deployment Options

#### 6.1.1 API Deployment

* Discuss the options for deploying the trained model as an API
* Explain the advantages and disadvantages of API deployment

#### 6.1.2 Web Application Deployment

* Discuss the options for deploying the trained model as a web application
* Explain the advantages and disadvantages of web application deployment

### 6.2 Model Serving

#### 6.2.1 Model Serving Options

* Discuss the options for serving the trained model (e.g., TensorFlow Serving, AWS SageMaker)
* Explain the advantages and disadvantages of each option

#### 6.2.2 Model Maintenance

* Explain the importance of model maintenance
* Discuss the methods used for model maintenance (e.g., model updating, model retraining)

## 7. Conclusion
--------------

### 7.1 Summary

* Summarize the key findings and results of the project
* Discuss the limitations and potential improvements of the project

### 7.2 Future Work

* Provide recommendations for future work
* Discuss the potential applications of the project